# 基础知识

让我们首先看一下“ib_async”包中的内容：

In [1]:
import ib_async
print(ib_async.__all__)

['util', 'Event', 'OrderComboLeg', 'OrderState', 'SoftDollarTier', 'PriceIncrement', 'Execution', 'CommissionReport', 'BarList', 'BarDataList', 'RealTimeBarList', 'BarData', 'RealTimeBar', 'HistogramData', 'NewsProvider', 'DepthMktDataDescription', 'ScannerSubscription', 'ScanDataList', 'FundamentalRatios', 'ExecutionFilter', 'PnL', 'PnLSingle', 'AccountValue', 'TickData', 'TickByTickAllLast', 'TickByTickBidAsk', 'TickByTickMidPoint', 'HistoricalTick', 'HistoricalTickBidAsk', 'HistoricalTickLast', 'TickAttrib', 'TickAttribBidAsk', 'TickAttribLast', 'MktDepthData', 'DOMLevel', 'TradeLogEntry', 'TagValue', 'FamilyCode', 'SmartComponent', 'PortfolioItem', 'Position', 'Fill', 'OptionComputation', 'OptionChain', 'Dividends', 'NewsArticle', 'HistoricalNews', 'NewsTick', 'NewsBulletin', 'ConnectionStats', 'Contract', 'Stock', 'Option', 'Future', 'ContFuture', 'Forex', 'Index', 'CFD', 'Commodity', 'Bond', 'FuturesOption', 'MutualFund', 'Warrant', 'Bag', 'ComboLeg', 'DeltaNeutralContract', 'Con

### 导入
以下两行代码位于所有笔记本的顶部。第一行导入所有内容，第二行启动一个事件循环以保持笔记本实时更新:

In [2]:
from ib_async import *
util.startLoop()

请注意，startLoop() 仅在笔记本中才能运行，而在常规的 Python 程序中无法运行。

### 连接
整个套件的主要角色是“IB”类。让我们创建一个IB实例并连接到正在运行的TWS/IBG应用程序：

In [3]:
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=10)

<IB connected to 127.0.0.1:7497 clientId=10>

如果连接失败，请确保应用程序已启用API端口，并仔细检查主机名和端口号。对于IB Gateway，默认端口是4002。请确保clientId尚未被使用。

如果连接成功，那么ib将与TWS/IBG同步。现在可以通过诸如ib.positions()、ib.trades()、ib.openTrades()、ib.accountValues()或ib.tickers()等方法获取“当前状态”。让我们列出当前持仓：

In [4]:
ib.positions()

[]

或者过滤账户价值以获得清算价值:

In [5]:
[v for v in ib.accountValues() if v.tag == 'NetLiquidationByCurrency' and v.currency == 'BASE']

[AccountValue(account='U172554', tag='NetLiquidationByCurrency', value='2039.4021', currency='BASE', modelCode='')]

“当前状态”将自动与TWS/IBG同步。因此，一旦订单被报告，订单填充将被添加，或者一旦TWS中的账户值发生变化，账户值将会被即时更新。

### 合同

合同可以以不同的方式指定：
* 使用 ibapi 的方式，通过创建一个空的 Contract 对象，并逐个设置其属性；
* 通过使用 Contract 并将属性作为关键字参数传递；
* 通过使用专门的 Stock、Option、Future、Forex、Index、CFD、Commodity、Bond、FuturesOption、MutualFund 或 Warrant 合同。

一些示例：

In [6]:
Contract(conId=270639)
Stock('AMD', 'SMART', 'USD')
Stock('INTC', 'SMART', 'USD', primaryExchange='NASDAQ')
Forex('EURUSD')
CFD('IBUS30')
Future('ES', '20180921', 'GLOBEX')
Option('SPY', '20170721', 240, 'C', 'SMART')
Bond(secIdType='ISIN', secId='US03076KAA60');

发送请求

IB类几乎包含了IB API提供的所有请求方法。返回结果的方法将会阻塞直到完成，然后返回结果。比如说，以reqContractDetails为例。

In [7]:
contract = Stock('TSLA', 'SMART', 'USD')
ib.reqContractDetails(contract)

[ContractDetails(contract=Contract(secType='STK', conId=76792991, symbol='TSLA', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='TSLA', tradingClass='NMS', comboLegs=[]), marketName='NMS', minTick=0.01, orderTypes='ACTIVETIM,ADJUST,ALERT,ALGO,ALLOC,AON,AVGCOST,BASKET,BENCHPX,COND,CONDORDER,DARKONLY,DARKPOLL,DAY,DEACT,DEACTDIS,DEACTEOD,DIS,GAT,GTC,GTD,GTT,HID,IBKRATS,ICE,IMB,IOC,LIT,LMT,LOC,MIDPX,MIT,MKT,MOC,MTL,NGCOMB,NODARK,NONALGO,OCA,OPG,OPGREROUT,PEGBENCH,POSTONLY,PREOPGRTH,REL,RPI,RTH,SCALE,SCALEODD,SCALERST,SNAPMID,SNAPMKT,SNAPREL,STP,STPLMT,SWEEP,TRAIL,TRAILLIT,TRAILLMT,TRAILMIT,WHATIF', validExchanges='SMART,AMEX,NYSE,CBOE,PHLX,ISE,CHX,ARCA,ISLAND,DRCTEDGE,BEX,BATS,EDGEA,CSFBALGO,JEFFALGO,BYX,IEX,EDGX,FOXRIVER,TPLUS1,NYSENAT,PSX', priceMagnifier=1, underConId=0, longName='TESLA INC', contractMonth='', industry='Consumer, Cyclical', category='Auto Manufacturers', subcategory='Auto-Cars/Light Trucks', timeZoneId='EST (Eastern Standard Time)', tradingHours

### 当前状态 vs 请求

进行请求涉及到网络流量的上行和下行，并且可能需要较长时间。另一方面，当前状态始终立即可用。因此，最好使用当前状态方法而不是请求。例如，优先使用``ib.openOrders()`` 而不是 ``ib.reqOpenOrders()``, 或者 ``ib.positions()`` 而不是 ``ib.reqPositions()``, 等等。

In [8]:
%time l = ib.positions()

CPU times: user 5 µs, sys: 3 µs, total: 8 µs
Wall time: 9.06 µs


In [9]:
%time l = ib.reqPositions()

CPU times: user 0 ns, sys: 745 µs, total: 745 µs
Wall time: 32.7 ms


### 日志记录

以下将在当前活动单元格下记录信息和更高级别的日志消息：

In [10]:
util.logToConsole()

查看所有调试消息（包括网络流量）：

In [11]:
import logging
util.logToConsole(logging.DEBUG)

### 断开连接

以下操作将断开连接``ib``，并清除其所有状态：

In [12]:
ib.disconnect()

2019-12-31 13:28:29,252 ib_async.ib INFO Disconnecting from 127.0.0.1:7497, 160 B sent in 9 messages, 21.0 kB received in 418 messages, session time 920 ms.
2019-12-31 13:28:29,255 ib_async.client INFO Disconnecting
